<a href="https://colab.research.google.com/github/karolinachorzepa/Dataanalysis_pandas_plotly_matplotlib_numpy/blob/master/AUM_Lab_1_1_Regresja_Notebook_with_R_14_11_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LAB 1.1 Algorytmy Uczenia Maszynowego**
v2021
## **Uczenie nadzorowane - regresja**
###Celem tego laboratorium jest:
- Praktyczne zapoznanie się z elementami modelu regresji oraz wpływem hiperparametrów na wynik szacunku parametrów.  
- Zdobycie umiejętności dopasowania linii regresji do danych i oceny modelu regresji w R
- Ogólne zaznajomnienie z jezykiem R w środowisku notatników Jupyter, które popularnie jest wykorzystywane w porotypowaniu i analityce Data Science.

###Plan:
- Instalacja i wczytanie potrzebnych bibliotek oraz danych
- Wstępne zapoznanie się z danymi oraz sformułowanie problemu
- Szacowanie parametrów modelu: spadek gradientowy, hiperparametry
- Predykcja i ewaluacja modelu

### **Intro**
Skopiuj notebook! **File > Save a copy in Drive**

You can have a simply empty R jupyter notebook in colab by using this URL link
https://colab.research.google.com/#create=true&language=r

Useful recap: https://www.roelpeters.be/running-an-r-kernel-in-google-colab/

In [ ]:
# Sprawdź wersję R
R.version.string

In [ ]:
text <- 'Hello, world!'
print(text)

In [ ]:
# Zainstaluj potrzebne pakiety
install.packages("caret")
install.packages("gridExtra")
install.packages("Metrics")
require(Metrics)
library(caret)
library("gridExtra")

In [ ]:
# Sprawdź czy dany pakiet jest zainstalowany
require('ggmap')
require('ggplot2')

Inicjujemy ziarno generatora liczb losowych, aby wyniki były odtwarzalne i za każdym razem takie same przy uruchomieniu skryptu.

In [ ]:
# Ziarno dla replikacji
set.seed(1234)

# **Uczenie nadzorowane**


### Zbiór danych do użycia: Boston Housing 

Źródło:
* Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. *J. Environ. Economics and Management* 5, 81–102.
* Belsley D.A., Kuh, E. and Welsch, R.E. (1980) *Regression Diagnostics. Identifying Influential Data and Sources of Collinearity*. New York: Wiley.

Dane dostępne publicznie w kilku miejscach: 
* Repozytorium danych [UCI](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data) 
* W ramach pakietu R MASS (Boston Massachusetts)
* Inne popularne strony dedykowane Data Science, np. [Kaggle](https://www.kaggle.com/c/boston-housing)

**The Boston data frame zawiera 506 wierszy i 14 kolumn:**
* crim - per capita crime rate by town.
* zn - proportion of residential land zoned for lots over 25,000 sq.ft.
* indus - proportion of non-retail business acres per town.
* chas - Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
* nox - nitrogen oxides concentration (parts per 10 million).
* rm - average number of rooms per dwelling.
* age - proportion of owner-occupied units built prior to 1940.
* dis - weighted mean of distances to five Boston employment centres.
* rad - index of accessibility to radial highways.
* tax - full-value property-tax rate per \$10,000.
* ptratio - pupil-teacher ratio by town.
* black - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
* lstat - lower status of the population (percent).
* medv - median value of owner-occupied homes in \$1000s.

In [ ]:
# Wczytaj dane z repozytorium internetowego
housing <- read.table("http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")
print(head(housing))

# Zmień/przypisz nazwy kolumn
names(housing) <- c('crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat', 'medv')
# Zobacz jak wyglądają dane, kilka wierszy od góry head() lub dołu tail()
head(housing)

In [ ]:
# Wczytaj dane z biblioteki R
require(MASS)
housing <- Boston

names(housing)

#### **Problem: Czy można przewidzieć wartość domu w zależności od liczby pokoi?**

In [ ]:
# Zobaczmy jak rozkładają się obserwacje w zbiorze danych
plot(housing$rm,housing$medv, col='black', main='Liczba pokoi a wartość domów')

In [ ]:
# Histogram mediany wartości (medv)

ggplot(housing, aes(x=medv)) + geom_histogram(color="black", fill="white",aes(y=..density..), position="identity", alpha=0.5) +
geom_density(alpha=0.6) 
# + labs(x = "medv", y = "Gęstość") 

In [ ]:
# hist(housing$rm)
ggplot(housing, aes(x=rm)) + geom_histogram(color="black", fill="white",aes(y=..density..), position="identity", alpha=0.5) +
geom_density(alpha=0.6)

In [ ]:
#---------------------------------------------------------
#Zbiór treningowy, walidacyjny i testowy: losowanie próbek
#---------------------------------------------------------
set.seed(1234)
# Losowanie z zastępowaniem. 
# Można też użyć createDataPartition() z biblioteki caret
index <- sample(3, nrow(housing), replace=TRUE, prob=c(0.6, 0.2, 0.2))
#head(ind)

# Compose training set
housing.train <- housing[index==1, 1:14]
# Compose validation set
housing.val <- housing[index==2, 1:14]
# Compose test set
housing.test <- housing[index==3, 1:14]

In [ ]:
# Sprawdzenie podziału procentowego
prop.table(table(index))

In [ ]:
hist(index)

In [ ]:
#------------------------------------------------------------------
# Częstą praktyką jest również osobne zapisanie zmiennej zależnej y 
# oraz macierzy zmiennych niezależnych x
#------------------------------------------------------------------

#Zbiór treningowy
y_train = housing.train$medv
x_train <- housing.train$rm
# Zbiór walidacyjny
y_val = housing.val$medv
x_val <- housing.val$rm
# Zbiór testowy
y_test = housing.test$medv
x_test <- housing.test$rm

In [ ]:
# Zobaczmy jak rozkładają się obserwacje w podzielonym zbiorze danych
ggplot(housing, aes(x=rm, y=medv)) + geom_point() +  facet_wrap(index)

### **1. Regresja liniowa - wstęp**

Regresja liniowa zakłada liniową zależność pomiędzy zmiennymi objaśniającymi i objaśnianymi. Zakłada zetem, że zmiana w $y$ jest proporcjonalna do zmiany w $x_{i}$.

$y = \theta_{0} +  \theta_{1}x_{1} + \theta_{2}x_{2} + ...$

Zadaniem regresji liniowej jest wyznaczenie parametrów $\theta$ tak, aby hipoteza była jak najbliższa prawdziwej funkcji dla przykładów trenujących $(𝑥, y)$. 

Najprostszy przypadek: model z jednym atrybutem:

$h(x|\theta) = \theta_{0} +  \theta_{1}x_{1} + ϵ$

#### **Problem: Czy można przewidzieć wartość domu w zależności od liczby pokoi?**

$medv = \theta_{0} +  \theta_{1}rm + ϵ$

### Budowanie i szacowanie parametrów modelu regresji na zbiorze treningowym:
* I Analitycznie: z układu równań normalnych $θ=(X^{T}X)^{−1}  X^{T}y$
* II Z pakietem R i dedykowaną funkcją: `lm()`

* III Przy pomocy algorytmu spadku gradientowego

### I Rozwiązanie analityczne, układ równań normalnych w zapisie wektorowym: 

$θ=(X^{T}X)^{−1}  X^{T}y$

In [ ]:
# Rozwiązanie analityczne: θ=(XTX)−1XTy
X <- cbind(1, matrix(x_train)) # Dodajemy kolumnę jedynek 
theta_a = solve((t(X)%*%X))%*%t(X)%*%y_train
theta_a

### II Regresja liniowa w R: lm()
Ogólna postać: `model <- lm(y_zmienna_wyjściowa ~ x_atrybut_1, zbiór_danych)` 

model: nazwa dowolna, często fit

Formuła hipotezy dla jednego atrybutu: `y ~ x)` 

Formuła hipotezy dla wielu atrybutów: `y ~ x1 + x2 + x3`


In [ ]:
# Regresja liniowa w R: dopasuj linię regresji 
fit <- lm( y_train ~ x_train ) 

# Pokaż strukturę modelu i oszacowane parametry
fit

In [ ]:
# Same wartości oszacowancyh parametrów
print(fit$coefficients)

In [ ]:
# Wartość theta1 - indeks zaczyna się od 1!
print(summary(fit)$coefficients[2,1]) 

In [ ]:
# Inne atrybuty dopasowanego modelu do przeglądania
print(attributes(fit))

In [ ]:
# Zbiór treningowy z dopasowaną linią regresji
plot(x_train,y_train, col=rgb(0.2,0.4,0.6,0.4), main='Dopasowana linia regresji')
# Nadpisz linię regresji
abline(fit, col='red')

### III Wyznaczanie parametrów regresji liniowej przy pomocy spadku gradientowego

**Błąd średniokwadratowy, funkcja kosztu**

![picture](https://3.bp.blogspot.com/-7vLgOxMhaBg/Tqjb34B9A9I/AAAAAAAADIg/w5E6cXN6x1c/s1600/cost_function.png)

In [ ]:
# Funkcja kosztu: błąd średniokwadratowy
cost <- function(X, y, theta) {
    # Wymiary macierzy danych:
    # X: m,len(theta)
    # y: m*1
    # theta: liczba parametrów*1
  sum((X %*% theta - y)^2) / (2*length(y))
}

**Hiperparametry** - to watrości które ustawiamy na początku i których wartość nie zmienia się w trakcie danego szacowania parametrów. 

In [ ]:
# Parametr szybkości uczenia - wielkość kroku
alpha <- 0.01

#Liczba iteracji: Ile kroków zejścia gradientowego?
num_iters <- 10000

# Inicjalizacja parametrów - przykładowo zera: h = 0 + 0*x 
theta <- matrix(c(0,0), nrow=2)

#---------------------------------------------------------
# Zachowanie historii uczenia
cost_history <- double(num_iters)
theta_history <- list(num_iters)
theta0_history <- double(num_iters)
theta1_history <- double(num_iters)

# add a column of 1's for the intercept coefficient
X <- cbind(1, matrix(x_train))

![picture](https://2.bp.blogspot.com/-AdV-O-MoZHE/TtLibFTaf9I/AAAAAAAAAVM/aOxUGP7zl98/s1600/gradient+descent+algorithm+OLS.png)

In [ ]:
# Spadek gradientowy - szacowanie parametrów hipotezy medv = theta0 +theta1*rm
for (i in 1:num_iters) {
  error <- (X %*% theta - y_train)
  delta <- t(X) %*% error / length(y_train)
  theta <- theta - alpha * delta       # Aktualizacja wartości parametrów
  cost_history[i] <- cost(X, y_train, theta)
  theta_history[[i]] <- theta
  theta0_history[i] <- theta[1,1]
  theta1_history[i] <- theta[2,1]
}

print('Oszacowane wartości parametrów theta przy danych wartościach: alpha, zainicjalizowanej theta i liczbie iteracji:')
print(theta)

### Jaka jest różnica z rozwiązaniem analitycznym?
* Zanotuj obserwacje.
* Jakie elementy mogły mieć na to wpływ i jak/w jakim stopniu?

In [ ]:
# Dane i stopniowa konwergencja dopasowania
plot(x_train,y_train, col=rgb(0.2,0.4,0.6,0.4), main='Regresja liniowa - zejście gradientowe i proces uczenia')
for (i in 1:num_iters) {
  abline(coef=theta_history[[i]], col=rgb(0.8,0,0,0.3))
}

# Linia regresji algorytmu zejścia gradientowego
abline(coef=theta, col='magenta', lwd = 5) 
legend("topleft", legend=c("Hipoteza", "Obserwacje"), lty=c(1,0), pch=c(NA, 16), col=c(rgb(0.8,0,0,0.3), rgb(0.2,0.4,0.6,0.4)))

# Linia regresji rozwiązania analitycznego (niebieska)
abline(coef=theta_a, col='blue', lwd = 5) 
legend("topleft", legend=c("Hipoteza", "Obserwacje", "Rozwiązanie analityczne"), lty=c(1,0,1), pch=c(NA, 16, NA), col=c(rgb(0.8,0,0,0.3), rgb(0.2,0.4,0.6,0.4), 'blue'))

In [ ]:
# Szczegóły eksperymentu
layout(matrix(c(1,2,3,4), 2, 2, byrow = TRUE))
plot(cost_history, type='o', col='red', lwd=2, main='Funkcja kosztu', ylab='Koszt', xlab='Liczba kroków (iteracji)', ylim = c(0,max(cost_history)))
plot(theta0_history, type='o', col='blue', lwd=2, main='Theta 0', ylab='Wartość theta0', xlab='Liczba kroków (iteracji)')
plot(theta1_history, type='o', col='blue', lwd=2, main='Theta 1', ylab='Wartość theta1', xlab='Liczba kroków (iteracji)')
plot(theta1_history, cost_history, col='green', lwd=2, main='Konwergencja wartości Theta 1', ylab='Wartość kosztu', xlab='Wartość theta1', xlim = c(1,18), ylim = c(0,max(cost_history)))
points(summary(fit)$coefficients[2,1], sum(summary(fit)$residuals^2)/(2*length(y_train)), pch=16, col='red')

In [ ]:
# 3D Plot (Loss Function, m, c) 
m <- seq(summary(fit)$coefficients[2,1] - 100, summary(fit)$coefficients[2,1] + 100, length=1000)  #seq(-100, 100, 5)
c <- seq(summary(fit)$coefficients[1,1] - 100, summary(fit)$coefficients[1,1] + 100, length=1000)  #seq(-100, 100, 5)
Y2<-sum(y_train^2)
X2<-sum(x_train^2)
XY<-sum(x_train*y_train)
X<-sum(x_train)
Y<-sum(y_train)
loss <- Y2+X2*m^2+c^2*length(y_train)+2*XY*m+2*Y*c-2*X*m*c
  
f <- function(m, c) {
     Y2+X2*m^2+c^2*length(y_train)+2*XY*m+2*Y*c-2*X*m*c
     }

z <- outer(m, c, f)
persp(m, c, z, phi = 30, theta = 30, col = "yellow", xlab = "m (Slope of the Line)", ylab = "c (Intercept on the Y-axis)", zlab = "Loss Function")

In [ ]:
# 2D Heat map (Loss Function, m, c) 
image(m,c,z,xlab = "m (Slope of the Line)",ylab = "c (Intercept on the Y-axis)",main="Loss Function Vs (m & c)")
par(new=TRUE)
contour(m,c,z, xaxt='n', yaxt='n',lwd = 2)
abline(h=1,col="blue")
abline(v=1,col="blue")

## **Ćwiczenie 1**

a) Zmień wartości parametru alpha i zanotuj obserwacje (np. 0.02, 0.03, 0.04, 0.001, 0.1, etc.)

b) Zmień liczbę iteracji i zanotuj obserwacje

c) Zmień wartości początkowe parametrów theta i zanotuj obserwacje

## **Ćwiczenie 2** 
Zaimplementuj regresję liniową dwóch lub więcej zmiennych i zanotuj obserwacje.

# **Predykcja i ocena modelu**

* Wytrenowaliśmy model.
* Można go teraz użyć do predykcji na nowych danych i jego ewaluacji.


**Ocena modelu:**
* in-sample error - widzieliśmy stopniowo malejący błąd na danych treningowych
* out-of-sample error - czy nasz model dobrze generalizuje?


## **Miary oceny modelu regresji liniowej I (RMSE)**

Czy wytrenowany model dobrze generalizuje?


Once we have a model, we use a "metric" to evaluate how well the model works. A metric is quantifiable and gives us an objective measure of how well the model predicts on new data. For regression problems, we will focus on:
$$RMSE = \sqrt{\frac{\sum_{i=1}^{N}(Predicted_{i} - Actual_{i})^{2}}{N}}$$

Funkcja `lm()` w R wykorzystuje RMSE jako metryki do minimalizacji.

Unfortunately, it's common practice to calculate RMSE on the same data we used to fit the model. This typically leads to overly-optimistic estimates of model performance. This is also known as overfitting. A better approach is to use out-of-sample estimates of model performance. 

However, it's useful to start off by looking at in-sample error, so we can contrast it later with out-of-sample error on the same dataset.

### In-sample - ewaluacja na całym zbiorze danych (treningowych) - explanatory modelling approach
Praktycznie gwarantuje przeuczenie

In [ ]:
# Fit lm model: model
model <- lm(medv ~ rm, housing.train)

# Predict on full data: p
predictions <- predict(model, housing.train)

# Compute errors: error
error <- predictions - housing.train[["medv"]]

# Calculate RMSE
sqrt(mean(error ^ 2))

### Out-of-sample validation

Teraz użyjemy danych walidacyjnych oraz modelu, który wytrenowaliśmy wcześniej na części danych w zbiorze treningowym (fit)

`model <- lm(y_train ~ x_train, training_data)`

aby uzyskać predykcje na nowych dancyh

`p <- predict(model, new_data) `

Focus here is on predictive modelling - cel to model który generalizuje dobrze na nowych danych. Key insight of machine learning. 

Pozwala na wybranie takiego modelu, który będzie dawał dobre wyniki w przyszłości na nowych danych

In [ ]:
# Uwaga na nazwy zmiennych w modelu i nowej porcji danych predykcyjnych

pred <- predict(model, newdata=housing.val)
summary(pred)

print(sqrt(mean((y_val-pred)^2))) # Błąd testowy (walidacyjny)

In [ ]:
# Predictions plot versus data
style <- c(rep(1,length(x_val)), rep(2,length(x_val)))
plot(c(x_val, x_val),c(y_val, pred),  xaxt="n", ylab="y_val and predicted", xlab="x_val", col=style, pch=1)
abline(fit)

In [ ]:
# Biblioteka ggplot2
# Zaznacz błędy
df <- data.frame(x = x_val, y = c(pred, y_val), Grupa = rep(c("Model: Predykcje", "Obserwacje"), each = length(x_val)))
ggplot(df, aes(x=x, y=y, group = Grupa)) +
  geom_point(aes(colour = Grupa), size = 3) + 
  geom_line(aes(group = x), linetype = 2) +
  geom_abline(intercept = summary(fit)$coefficients[1,1], slope = summary(fit)$coefficients[2,1]) +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
    panel.background = element_blank(), axis.line = element_line(colour = "black"))

In [ ]:
# Błąd średniokwadratowy: MSE
print('Błąd treningowy średniokwadratowy: MSE')
print(mean(fit$residuals^2))

# RMSE (od ang. root mean square error) czyli średnia kwadratowa błędów (wariancja + obciążenie), który jest po prostu pierwiastkiem kwadratowym z MSE.
print('Pierwiastek błędu średniokwadratowego: RMSE na zbiorze treningowym')
print(sqrt(mean(fit$residuals^2))) # Błąd treningowy
print('Pierwiastek błędu średniokwadratowego: RMSE na zbiorze walidacyjnym')
print(sqrt(mean((y_val-pred)^2))) # Błąd testowy (walidacyjny)

### Walidacja krzyżowa (Cross-validation)

W poprzednim pojejściu, ewaluacja miała miejce na nowych dancyh tylko raz. Jeden duży outlier może nam sporo zepsuć. Model performance is dependent on way the data is split. Not representative of the model’s ability to generalize.

 a better approach to validating models is to use multiple systematic test sets, rather than a single random train/test split. Solution: Cross-validation!

Tutaj Obliczamy metryki na foldach, dzieląc zbiór na kilka train-test zborów i uśredniając out of sample error. Cel: bycie bardziej precyzyjnym. Każda obserwacja pojawia się tylko w jednym zbiorze testowym. 

Ale to podejście jest bardzo drogie! (Obliczeniowo)

Pakiet `caret` umożliwia łatwą implementację walidacji krzyzowej. 

In [ ]:
set.seed(1234)

# Fit liner regression model
# model <- train(y ~ ., my_data)

model <- train(
  medv ~ rm, 
  housing.train,
  method = "lm",
  trControl = trainControl(
    method = "cv", 
    number = 10,
    verboseIter = TRUE
  )
)

# Print model to console
model

In [ ]:
set.seed(1234)

# You can do more than one iteration of cross validation! 5 x 5-fold cross-validation
# method: repeatedcv

# Fit lm model using 5 x 5-fold CV: model
model <- train(
  medv ~ rm, 
  housing.train,
  method = "lm",
  trControl = trainControl(
    method = "repeatedcv", 
    number = 5,
    repeats = 5, 
    verboseIter = TRUE
  )
)

# Print model to console
model

In [ ]:
# Fianl evaluation on new test data!

# Uwaga na nazwy zmiennych w modelu i nowej porcji danych predykcyjnych

predtest <- predict(fit, newdata=data.frame(x_train=x_test))
summary(predtest)

print(sqrt(mean((y_test-predtest)^2))) # Błąd testowy (walidacyjny)

## **Miary oceny modelu regresji liniowej II $R^{2}$**
* R-Squared is determined from the score method of the regression object.
* For Regression, we are going to use the coefficient of determination as our way of evaluating the results, also referred to as R-Squared

![picture](https://docs.oracle.com/cd/E12825_01/epm.111/cb_statistical/images/graphics/r_squared_constant.gif)



In [ ]:
# Błąd testowy przy użyciu biblioteki
print(rmse(y_val,pred)) 

In [ ]:
# R-squared 
# Adjusted R-squared - corrects for the number of features
summary(fit)$r.squared 

### Podsumowanie modelu w R: `summary(nazwa_modelu)`
Wyświetla szczegółowe informacje na temat dopasowania modelu i oszacowanych parametrów.

In [ ]:
summary(fit)

#### **Interpretacja składowych podsumowania modelu regresji liniowej**

![picture](https://www.ucl.ac.uk/~uctqiax/PUBLG100/2015/img/lm.png)

## **Założenia regresji (wybrane)**

### Założenie: The regression model is linear in parameters
### Założenie: The regression model is correctly specified

In [ ]:
# Założenie: Homoscedasticity of residuals or equal variance

# Wizualna diagnostyka modelu
#layout(matrix(c(1,2,3,4), 2, 2, byrow = TRUE))
par(mfrow=c(2,2)) 
plot(fit)

In [ ]:
# Założenie: No perfect multicollinearity between explanatory variables

# Variance Inflation factor (VIF). But, What is VIF?
# VIF is a metric computed for every X variable that goes into a linear model. 
# If the VIF of a variable is high, it means the information in that variable is already explained by other X variables present in the given model, which means, more redundant is that variable. 
# So, lower the VIF (<2) the better. VIF for a X var is calculated as:
#                         VIF=1/(1−Rsq)
#vif(model)

1/(1-summary(fit)$r.squared)

In [ ]:
# Założenie 10: Normality of residuals

par(mfrow=c(2,2))
plot(fit)

# Podsumowanie
W tym ćwiczeniu poznaliśmy zagadnienia związane z regresją liniową na przykładzie danych Boston housing. 

Zachęcam do przećwiczenia tych aspektów na innych zbiorach danych. 